# Artificial Art with more than one style image

In diesem Projekt wollen wir uns mit dem Erstellen von Bildern aus Content- und Style- Bildern mithilfe von maschinellem Lernen beschäftigen. 

Als Grundlage dieser Arbeit dient [dieses Notebook von Harish Narayanan](https://github.com/hnarayanan/artistic-style-transfer/blob/master/notebooks/6_Artistic_style_transfer_with_a_repurposed_VGG_Net_16.ipynb).
Wir erweitern dieses um unsere eigenen Gedanken, und führen die Option ein, mehrere Style- Bilder zu verwenden. Am Ende zeigen wir unsere eigenen Bilder und welchen Einfluss die verschiedenen Parameter haben.

Als Frameworks nutzen wir `keras`, welches Tensorflow (1.0) als Backend benutzt, sowie `scypy` für einen Optimierungsalgorithmus sowie `PIL`, um Bilder laden und bearbeiten zu können.

In [1]:
import numpy as np
import copy

import time
from PIL import Image

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

from scipy.optimize import fmin_l_bfgs_b

Using TensorFlow backend.


Zunächst geben wir die Größe der Bilder an. Egal, welche Größe die Ausgangsbilder haben, sie werden alle auf diese Größe skaliert.

Um maschinelles Lernen betreiben zu können, werden die Bilder in Numpy-Arrays konvertiert. Es wird eine Dimension hinzugefügt, damit am Ende alle Bilder in einem Datenobjekt zusammengefasst werden können.

Wir werden das viel verwendete VGG16-Modell benutzen. Da dieses Modell ein spezielles Datenformat verlangt, müssen die Daten noch folgendermaßen angepasst werden:
* Von allen Pixeln werden die mittleren RGB-Werte in den entsprechenden Farbchannels abgezogen.
* Die Farbchannels werden in ihrer Reihenfolge umgekehrt.

In [19]:
height = 512
width = 512

content_image_paths = [ "images/bild_content_quadratisch.jpg" ]
content_imgs = []
for content_img_path in content_image_paths:
    content_img = Image.open(content_img_path)
    content_img = content_img.resize((height, width))
    content_arr = np.asarray(content_img, dtype='float32')
    content_arr = np.expand_dims(content_arr, axis=0)
    content_arr[:, :, :, 0] -= 103.939
    content_arr[:, :, :, 1] -= 116.779
    content_arr[:, :, :, 2] -= 123.68
    # Convert from RGB to BGR
    content_arr = content_arr[:, :, :, ::-1]
    content_img = backend.variable(content_arr)
    content_imgs.append(content_img)

In [20]:
style_image_paths = ['images/styles/wave.jpg']
style_imgs = []
for style_img_path in style_image_paths:
    style_img = Image.open(style_img_path)
    style_img = style_img.resize((height, width))
    style_arr = np.asarray(style_img, dtype='float32')
    style_arr = np.expand_dims(style_arr, axis=0)
    style_arr[:, :, :, 0] -= 103.939
    style_arr[:, :, :, 1] -= 116.779
    style_arr[:, :, :, 2] -= 123.68
    # Convert from RGB to BGR
    style_arr = style_arr[:, :, :, ::-1]
    style_img = backend.variable(style_arr)
    style_imgs.append(style_img)

In unserem Falle sind wir nicht daran interessiert, das Klassifikationsnetzwerk zur tatsächlichen Klassifikation zu benutzen, sondern wollen statt Weights und Biases anzupassen unsere Inputs anpassen, um die Kostenfunktion zu minimieren (diese wird später genauer erläutert). Dafür brauchen wir in den Inputs neben den Ausgangsbildern noch Platz für das Ergebnisbild. Dafür erstellen wir einen Placeholder, der dieselben Dimensionen hat.

In [4]:
# Channels as the last dimension, using backend Tensorflow
combination_img = backend.placeholder((1, height, width, 3))

Wir konkatenieren alle Ausgangsbilder und den Platzhalter für das Ergebnisbild in einen Tensor, der dem Modell als Input dient.

In [5]:
all_imgs = []
for content_img in content_imgs:
    all_imgs.append(content_img)
for style_img in style_imgs:
    all_imgs.append(style_img)
all_imgs.append(combination_img)
input_tensor = backend.concatenate(all_imgs, axis=0)

Nun wird das Netzwerk initialisiert. Genaueres zum Aufbau in [diesem Notebook](VGG16_mnist.ipynb). Wir nutzen das bereits mit Daten von Imagenet trainierte Netzwerk und lassen die Fully-Connected-Layers weg, da diese nur für die Klassifikation benötigt werden und keine für uns relevanten Informationen enthalten.

In [6]:
model = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)

In [7]:
layers = dict([(layer.name, layer.output) for layer in model.layers])
layers

{'block1_conv1': <tf.Tensor 'block1_conv1/Relu:0' shape=(3, 512, 512, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2/Relu:0' shape=(3, 512, 512, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool/MaxPool:0' shape=(3, 256, 256, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1/Relu:0' shape=(3, 256, 256, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2/Relu:0' shape=(3, 256, 256, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool/MaxPool:0' shape=(3, 128, 128, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1/Relu:0' shape=(3, 128, 128, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2/Relu:0' shape=(3, 128, 128, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3/Relu:0' shape=(3, 128, 128, 256) dtype=float32>,
 'block3_pool': <tf.Tensor 'block3_pool/MaxPool:0' shape=(3, 64, 64, 256) dtype=float32>,
 'block4_conv1': <tf.Tensor 'block4_conv1/Relu:0' shape=(3, 64, 64, 512) dtype=float32>,
 'b

Wie man sehen kann, schlägt sich die Bildgröße in allen Schichten in der Parameteranzahl wider. Dies bedeutet, dass eine Erhöhung der Auflösung zu einer deutlich überproportional längeren Berechnungsdauer führt.

Um ein möglichst gutes Endergebnis zu erhalten, muss dafür gesorgt werden, dass sowohl der Inhalt der Content-Bilder erhalten bleibt, als auch der Style der Style-Bilder angemessen im Ergebnisbild zu erkennen ist. Da diese mit einander in Konflikt stehen, nutzen wir als Loss eine gewichtete Summe. 

Im Folgenden werden die einzelnen Bestandteile der Kostenfunktion genauer erläutert. 

Die Werte der Gewichte haben wir aus [Narayanans Notebook](https://github.com/hnarayanan/artistic-style-transfer/blob/master/notebooks/6_Artistic_style_transfer_with_a_repurposed_VGG_Net_16.ipynb) übernommen. Durch viel Ausprobieren fand er diese Werte, die zu einem subjektiv als gut empfundenen Ergebnis geführt haben. Es bedeutet nicht, dass der Style 200x wichtiger als der Content ist, da die Ergebnisse der einzelnen Loss-Funktionen nicht auf eine Verteilung normalisiert werden und somit nicht direkt miteinander vergleichbar sind.

In [8]:
content_weight = 0.025
style_weight = 5.0
tv_weight = 1.0

In [9]:
loss = backend.variable(0.)

Der Content-Loss ist die euklidische Distanz zwischen den Outputs einer ausgewählten Schicht für den Content und unser Ergebnisbild. 

Die höheren Schichten erkennen eher größere Strukturen und weniger konkrete Details.

In [10]:
def content_loss(content, combination):
    return backend.sum(backend.square(combination - content))

# Add content loss to this layer
layer_features = layers['block2_conv2']

content_features = layer_features[0, :, :, :]
combination_features = layer_features[len(content_imgs) + len(style_imgs), :, :, :]

loss += content_weight * content_loss(content_features, combination_features)



Der Style-Loss ist der interessanteste Teil dieses Algorithmus. Hier wird eine sogenannte Gram-Matrix zwischengeschaltet, die die Korrelation der Outputs der Neuronen verschiedener Schichten für alle Style-Bilder repräsentiert. Somit werden Informationen über die einzelnen Outputs verloren, aber Informationen über die Zusammenhänge der Neuronen untereinander gewonnen. Dies ist ein erstaunlich guter Indikator für Stil-Ähnlichkeit.

In der Gram Matrix werden alle Outputs der betroffenen Neuronen paarweise miteinander multipliziert. Dafür wird zunächst ein Spaltenvektor aus allen n Outputs generiert und dieser mit seiner Transposition multipliziert, sodass eine n x n - Matrix entsteht.

In [11]:
def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = height * width
    return backend.sum(backend.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

feature_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3']
for layer_name in feature_layers:
    layer_features = layers[layer_name]
    for style_img_idx in range(len(style_imgs)):
        style_features = layer_features[1 + style_img_idx, :, :, :]
        combination_features = layer_features[len(content_imgs) + len(style_imgs), :, :, :]
        style_l = style_loss(style_features, combination_features)
        loss += (style_weight / (len(feature_layers)*len(style_imgs))) * style_l

Damit das Ergebnisbild nicht zu verrauscht ist, wird noch ein dritter Term in die Loss-Funktion aufgenommen: der Total Variation Loss. Hierbei wird einmal das Bild ohne letzte Zeile über das Bild ohne erste Zeile gelegt und die euklidische Distanz berechnet. Für die Spalten wird analog vorgegangen und das Ergebnis aufaddiert. So wird insgesamt jeder Pixel einmal mit jedem direkten Nachbarn (oben, unten, rechts, links) verglichen und zu große Unterschiede werden bestraft. Somit wird einem zu großen Rauschen entgegengewirkt.

In [12]:
# Total variation loss to ensure the image is smooth and continuous throughout

def total_variation_loss(x):
    a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))

loss += tv_weight * total_variation_loss(combination_img)

Im Folgenden findet der Lernprozess statt. Wichtig ist, dass die Variablen `loss` (von oben: `backend.variable`), `grads` und `f_outputs` die jeweiligen `backend`-Funktionen nutzen, damit sie in jeder Iteration neu evaluiert werden können. Der Evaluator wurde als Klasse implementiert, damit die Funktion `fmin_l_bfgs_b` in der for-Schleife die Werte von `loss` und `grads` an zwei Stellen übergeben werden können, ohne zweimal berechnet werden zu müssen (sie werden nur bei `Evaluator.loss()` berechnet)

In [13]:
grads = backend.gradients(loss, combination_img)

In [14]:
outputs = [loss]
outputs += grads
# Create the function from input combination_img to the loss and gradients
f_outputs = backend.function([combination_img], outputs)

In [15]:
# We finally have the gradients and losses at the combination_img computed as variables
# and we can use any standard optimization function to optimize combination_img

def eval_loss_and_grads(x):
    x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [16]:
# random first image
x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.0

# in order for the output to be readable for humans, 
# the transformations done in the beginning have to be undone before saving.
def transform_back(x):
    x1 = copy.deepcopy(x)
    x1 = x1.reshape((height, width, 3))
    # Convert back from BGR to RGB to display the image
    x1 = x1[:, :, ::-1]
    x1[:, :, 0] += 103.939
    x1[:, :, 1] += 116.779
    x1[:, :, 2] += 123.68
    x1 = np.clip(x1, 0, 255).astype('uint8')
    return Image.fromarray(x1)

transform_back(x).save('Galerie/before.bmp')

iters = 10

for i in range(iters):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

    transform_back(x).save('Galerie/result' + str(i) + '.bmp')

('Start of iteration', 0)
('Current loss value:', 3.4901606e+10)
Iteration 0 completed in 1065s
('Start of iteration', 1)
('Current loss value:', 1.8784664e+10)
Iteration 1 completed in 879s
('Start of iteration', 2)
('Current loss value:', 1.2682997e+10)
Iteration 2 completed in 905s
('Start of iteration', 3)
('Current loss value:', 1.0791149e+10)
Iteration 3 completed in 902s
('Start of iteration', 4)
('Current loss value:', 9.7563412e+09)
Iteration 4 completed in 872s
('Start of iteration', 5)
('Current loss value:', 9.148119e+09)
Iteration 5 completed in 918s
('Start of iteration', 6)
('Current loss value:', 8.7887421e+09)
Iteration 6 completed in 909s
('Start of iteration', 7)
('Current loss value:', 8.5374116e+09)
Iteration 7 completed in 942s
('Start of iteration', 8)
('Current loss value:', 8.3602642e+09)
Iteration 8 completed in 907s
('Start of iteration', 9)
('Current loss value:', 8.2301092e+09)
Iteration 9 completed in 898s


Im Folgenden wollen wir auf die Ergebnisse für verschiedene Parameterwerte eingehen. Wir haben jeweils alle Werte so gelassen wie oben zu sehen, und einen Wert verändert, um deren Einfluss zu studieren. Links ist immer ein gif zu sehen, welches die Entwicklung zeigt, und rechts das Ergebnisbild der letzten Iteration. Für einen besseren Gesamtüberblick sind die Bilder hier verkleinert dargestellt. Im Galerie-Ordner kann man sie sich in 512x512 anschauen.

Folgende Bilder haben wir als Inputs genommen:
    
Content: <img src="images/bild_content_quadratisch.jpg" alt="content" width="256"/> Style: <img src="images/styles/wave.jpg" alt="content" width="256"/>

___

Zunächst das Bild mit den Standard-Parametern:

<img src="Galerie/standard/result.gif" width="256"/>    <img src="Galerie/standard/result9.bmp" width="256"/>

<!---
![standard](Galerie/standard/result.gif) ![standard](Galerie/standard/result9.bmp)
-->
Wie man sieht, ist das Ergebnis ein guter Kompromiss zwischen allen Parametern.
___

Zwei Styles:

<img src="Galerie/1_content_2_styles/result.gif" width="256"/>    <img src="Galerie/1_content_2_styles/result7.bmp" width="256"/>

<!---
![standard](Galerie/1_content_2_styles/result.gif) ![standard](Galerie/1_content_2_styles/result7.bmp)
-->
___

Hier wurden zwei Style-Bilder übergeben. Zum einen, dass im Standard definierte und als zweites dieses hier:

<img src="images/styles/marilyn.jpg"  width="256"/>

`content_weight = 0.5 # Standard: 0.025`

<img src="Galerie/content_weight_0-5/result.gif" width="256"/>    <img src="Galerie/content_weight_0-5/result9.bmp" width="256"/>

<!---
![standard](Galerie/content_weight_0-5/result.gif) ![standard](Galerie/content_weight_0-5/result9.bmp)
-->
Hier kommt der Style kaum zur Geltung. Er schlägt sich fast nur in den Farbtönen nieder.
___

`style_weight = 15 # Standard: 5`

<img src="Galerie/style_weight_15/result.gif" width="256"/>    <img src="Galerie/style_weight_15/result9.bmp" width="256"/>

<!---
![standard](Galerie/style_weight_15/result.gif) ![standard](Galerie/style_weight_15/result9.bmp)
-->
Hier kommt auch ein sehr ansprechendes, noch verzierteres Bild heraus.
___

`tv_weight = 0 # Standard: 1`

<img src="Galerie/tv_weight_0/result.gif" width="256"/>    <img src="Galerie/tv_weight_0/result9.bmp" width="256"/>

<!---
![standard](Galerie/tv_weight_0/result.gif) ![standard](Galerie/tv_weight_0/result9.bmp)
-->
Das Rauschen aus dem Anfangs-Zufallsbild wird ohne total-variation-loss wenig abgemildert, sodass das Ergebnis sehr verrauscht ist.
___

`tv_weight = 10 # Standard: 1`

<img src="Galerie/tv_weight_10/result.gif" width="256"/>    <img src="Galerie/tv_weight_10/result9.bmp" width="256"/>

<!---
![standard](Galerie/tv_weight_10/result.gif) ![standard](Galerie/tv_weight_10/result9.bmp)
-->
Hier entstehen viele klar abgegrenzte Flächen gleicher Farbe und weniger Farbübergänge. Der Himmel zeigt eigenartigerweise mehr Struktur als im Standard-Bild.
___

`tv_weight = 100 # Standard: 1`

<img src="Galerie/tv_weight_100/result.gif" width="256"/>    <img src="Galerie/tv_weight_100/result9.bmp" width="256"/>

<!---
![standard](Galerie/tv_weight_100/result.gif) ![standard](Galerie/tv_weight_100/result9.bmp)
-->
Hier wird das Rauschen so stark verhindert, dass das im Ergebnis kaum Feinheiten zu erkennen sind, sondern nur noch größere Farbpunkte.
___

`layer_features = layers['block1_conv1'] # Standard: layers['block2_conv2']`

<img src="Galerie/layer_features_block1_conv1/result.gif" width="256"/>    <img src="Galerie/layer_features_block1_conv1/result9.bmp" width="256"/>

<!---
![standard](Galerie/layer_features_block1_conv1/result.gif) ![standard](Galerie/layer_features_block1_conv1/result9.bmp)
-->
Im ersten convolutional layer scheinen die vom Netzwerk erkannten Content-Strukturen noch so detailliert zu sein, dass hier für uns Menschen nichts zu erkennen ist.
___

`layer_features = layers['block3_conv1'] # Standard: layers['block2_conv2']`

<img src="Galerie/layer_features_block3_conv1/result.gif" width="256"/>    <img src="Galerie/layer_features_block3_conv1/result9.bmp" width="256"/>

<!---
![standard](Galerie/layer_features_block3_conv1/result.gif) ![standard](Galerie/layer_features_block3_conv1/result9.bmp)
-->
Dieses Ergebnis ist dem Standard sehr ähnlich. Es ist ein ähnlicher Effekt, wie den style_weight zu erhöhen. Dies lässt darauf schließen, dass hier der das Netzwerk schon etwas abstraktere Konzepte verwendet, die für uns Menschen den Content etwas schlechter erkennen lassen. Die folgenden Bilder bestätigen dies.
___

`layer_features = layers['block4_conv1']:`

<img src="Galerie/layer_features_block4_conv1/result.gif" width="256"/>    <img src="Galerie/layer_features_block4_conv1/result9.bmp" width="256"/>

<!---
![standard](Galerie/layer_features_block4_conv1/result.gif) ![standard](Galerie/layer_features_block4_conv1/result9.bmp)
-->
Hier ist der Content nur noch schemenhaft zu erkennen.
___

`layer_features = layers['block5_conv3']:`

<img src="Galerie/layer_features_block5_conv3/result.gif" width="256"/>    <img src="Galerie/layer_features_block5_conv3/result9.bmp" width="256"/>

<!---
![standard](Galerie/layer_features_block5_conv3/result.gif) ![standard](Galerie/layer_features_block5_conv3/result9.bmp)
-->
In der letzten convolutional Layer scheinen die Neuronen so abstrakte Zusammenhänge zu repräsentieren (am Ende muss das Netzwerk Objekte aus allen möglichen Variationen von Winkel, Lichtverhältnissen, teilweise Verdeckung etc. erkennen können), dass bei Verwendung dieser Layer als Content-Indikator für uns Menschen nichts mehr zu erkennen ist. In einem zukünftigen Projekt könnte man untersuchen, ob das Netzwerk bei Verwendung zur Klassifizierung hier dennoch z.B. zwei menschliche Gesichter erkennen würde.
___

`feature_layers = layers['block1_conv1, block_2_conv1, block3_conv1]:`

<img src="Galerie/feature_layers/result.gif" width="256"/>    <img src="Galerie/feature_layers/result9.bmp" width="256"/>

<!---
![standard](Galerie/feature_layers/result.gif) ![standard](Galerie/feature_layers/result9.bmp)
-->

Auch bei den style-Layern wollten wir weitere Variationen ausprobieren. Anstatt aus jedem Block den letzten conv-Layer zu verwenden, haben wir hier aus den ersten drei Blöcken den ersten conv-Layer verwendet. Zu erkennen ist, dass die Farben des style-Bildes richtig erkennt werden sowie auch schon einzelne Strukturen.

`feature_layers = layers['block4_conv3, 'block5_conv3']`

<img src="Galerie/feature_layers/block4_conv3,block5_conv3/result.gif" width="256"/>    <img src="Galerie/feature_layers/block4_conv3,block5_conv3/result9.bmp" width="256"/>

Hier werden zu wenige style-Layers verwendet, weswegen im Bild eher wenig des Stylebildes übernommen wird, bis  auf das farbliche im Himmel und im Wasser. Somit lässt sich daraus schließen, dass mindestens 3 Layer verwendet werden sollten und mehr noch einen besseres Resultat in Hinsicht des Styles generiert wird.